In [182]:
import pandas as pd
import numpy as np
import re

Load data frame from csv file

In [183]:
data_df = pd.read_csv('example_dataframe_100_recipes.csv')
data_df.head(10)

,recipe_title,cuisine_SP,aisle_SP,ingredients_SP
0,unknown,[],"['Produce', 'Milk, Eggs, Other Dairy', 'Spices...","['cauliflower', 'egg', 'italian seasoning', 'm..."
1,Dreamy Chai Rice Pudding,[],"['Pasta and Rice', 'Milk, Eggs, Other Dairy', ...","['long grain rice', 'whole milk', 'tea', 'cinn..."
2,Instant Pot Chicken Tacos,['Mexican'],"['Spices and Seasonings', 'Meat', 'Produce', '...","['black pepper', 'chicken breasts', 'lime juic..."
3,Homemade King Ranch Chicken Casserole,[],"['Milk, Eggs, Other Dairy', 'Meat', 'Canned an...","['butter', 'cooked chicken breasts', 'cream of..."
4,Mangoes with Rum and Ice Cream,[],"['Produce', 'Baking', 'Alcoholic Beverages', '...","['mangoes', 'sugar', 'dark rum', 'lime juice',..."
5,Trinidadian Chicken Potato Curry,"['Indian', 'Asian']","['Meat', 'Produce', 'Produce', 'Produce;Spices...","['bone in chicken thighs', 'yellow onion', 'ga..."
6,Falafel Burgers,['American'],"['Canned and Jarred', 'Spices and Seasonings',...","['canned garbanzo beans', 'chili powder', 'cor..."
7,Delicious Rhubarb Custard Pie,[],"['Milk, Eggs, Other Dairy', 'Milk, Eggs, Other...","['butter', 'eggs', 'flour', 'nutmeg', 'pie cru..."
8,Key Lime Pie,['Mexican'],"['Baking', 'Produce', 'Produce', 'Milk, Eggs, ...","['graham cracker crust', 'juice of lime', 'lim..."
9,No-Bake Fudge Brandy Brownies,['American'],"['Alcoholic Beverages', 'Milk, Eggs, Other Dai...","['brandy', 'butter', 'condensed milk', 'oreo',..."


Create a new data frame with only Mexican recipes

In [184]:
mexican_df = data_df[data_df['cuisine_SP'] == "['Mexican']"].copy()
mexican_df

,recipe_title,cuisine_SP,aisle_SP,ingredients_SP
2,Instant Pot Chicken Tacos,['Mexican'],"['Spices and Seasonings', 'Meat', 'Produce', '...","['black pepper', 'chicken breasts', 'lime juic..."
8,Key Lime Pie,['Mexican'],"['Baking', 'Produce', 'Produce', 'Milk, Eggs, ...","['graham cracker crust', 'juice of lime', 'lim..."
12,Enchiladas Verdes (Green Enchiladas),['Mexican'],"['Meat', 'Produce;Ethnic Foods;Spices and Seas...","['skinless boneless chicken breast', 'chili pe..."
45,Salsa Verde Chicken Tamales,['Mexican'],"['Meat', 'Meat', 'Produce', 'Produce', 'Spices...","['roasted chicken', 'chicken', 'garlic', 'onio..."
89,Mexican Casserole,['Mexican'],"['Meat', 'Cheese', 'Canned and Jarred', 'Canne...","['chicken breasts', 'colby jack', 'cream of ch..."


For each row (recipe) remove the unwanted characters from the ingredients string, split the string at commas to return a list, save the returned list back to the dataframe.  Also create a running list of all elements used in the recipes

In [185]:
def ingredients_cleanup (data_df, column):
    total_ingredients_list_mex = []
    row_list_mexican = []
    indexes = data_df.index.values.tolist()
    counter = 0
    for row in data_df[column]:
        row = row.replace("[", '')
        row = row.replace("'", '')
        row = row.replace("]", '')
        row = row.replace('"', '')
        row = re.sub(r"^\s+" , "" , row)
        row_list_mexican = row.split( ',')
        data_df.loc[indexes[counter],column] = row_list_mexican
        counter += 1
    return data_df

In [186]:
mexican_df = ingredients_cleanup(mexican_df, 'ingredients_SP')
mexican_df = ingredients_cleanup(mexican_df, 'aisle_SP')


In [187]:
mexican_df.head()

,recipe_title,cuisine_SP,aisle_SP,ingredients_SP
2,Instant Pot Chicken Tacos,['Mexican'],"[Spices and Seasonings, Meat, Produce, Prod...","[black pepper, chicken breasts, lime juice, ..."
8,Key Lime Pie,['Mexican'],"[Baking, Produce, Produce, Milk, Eggs, Ot...","[graham cracker crust, juice of lime, lime z..."
12,Enchiladas Verdes (Green Enchiladas),['Mexican'],"[Meat, Produce;Ethnic Foods;Spices and Season...","[skinless boneless chicken breast, chili pepp..."
45,Salsa Verde Chicken Tamales,['Mexican'],"[Meat, Meat, Produce, Produce, Spices and ...","[roasted chicken, chicken, garlic, onion, ..."
89,Mexican Casserole,['Mexican'],"[Meat, Cheese, Canned and Jarred, Canned an...","[chicken breasts, colby jack, cream of chick..."


Use the composite list of ingredients to determine the 20 most common ingredients in the cuisine.

In [188]:
type(mexican_df.ingredients_SP[2])

list

In [189]:
composite_ingredients_mexican = []
for row in mexican_df.ingredients_SP:
    for element in row:
        composite_ingredients_mexican.append(element)
print(composite_ingredients_mexican)

['black pepper', ' chicken breasts', ' lime juice', ' purple onion', ' salsa', ' taco seasoning', 'graham cracker crust', ' juice of lime', ' lime zest', ' sour cream', ' sweetened condensed milk', ' whipped cream', 'skinless boneless chicken breast', ' chili peppers', ' canola oil', ' buttermilk', ' flour', ' white onion', ' white corn tortillas', ' cheese', ' queso fresco', ' crema', ' salt', ' water', 'roasted chicken', ' chicken', ' garlic', ' onion', ' cumin', ' sea salt', ' olive oil', ' water', ' salsa verde', ' tomatillos', ' chile', ' chile', ' chiles', ' yellow onion', ' cilantro', ' sea salt', ' filo dough', ' masa flour', ' masa flour', ' sea salt', ' palm oil', ' chicken broth', ' salsa verde', 'chicken breasts', ' colby jack', ' cream of chicken soup', ' cream of mushroom soup', ' onion', ' rice', ' salsa', ' shredded cheddar cheese']


Create a dataframe from the composite ingredients list so it can be analyzed.

In [190]:
def sort_and_count_ingredient_list(ingredient_list):
    ingredient_df = pd.DataFrame(ingredient_list)
    ingredient_df = ingredient_df.rename(columns = {0:  'ingredient'})
    ingredient_df['count'] = 1
    ingredient_counts = ingredient_df.groupby('ingredient').agg({'count':  'count'})
    ingredient_counts.sort_values('count', ascending=False, inplace=True)
    return ingredient_counts


In [191]:
mexican_ing_groups = sort_and_count_ingredient_list(composite_ingredients_mexican)
mexican_ing_groups.head(20)

,count
ingredient,
sea salt,3
onion,2
masa flour,2
salsa,2
salsa verde,2
chile,2
water,2
taco seasoning,1
rice,1


Create a dictionary to use to replace different names for the same ingredients.  In final product create reg ex logic to make these changes

In [192]:
replacement_dict = {'sea salt':  'salt_and_pepper',
                    'salsa verde':  'salsa',
                    'chicken breasts':  'chicken',
                    'purple onion':  'onion',
                    'roasted chicken':  'chicken',
                    'white onion':  'onion',
                    'salt':  'salt_and_pepper',
                    'skinless boneles chicken breast':   'chicken'
}
replacement_dict.keys()

dict_keys(['sea salt', 'salsa verde', 'chicken breasts', 'purple onion', 'roasted chicken', 'white onion', 'salt', 'skinless boneles chicken breast'])

In [193]:
mexican_df.replace(to_replace=replacement_dict, inplace=True)

In [194]:
mexican_df

,recipe_title,cuisine_SP,aisle_SP,ingredients_SP
2,Instant Pot Chicken Tacos,['Mexican'],"[Spices and Seasonings, Meat, Produce, Prod...","[black pepper, chicken breasts, lime juice, ..."
8,Key Lime Pie,['Mexican'],"[Baking, Produce, Produce, Milk, Eggs, Ot...","[graham cracker crust, juice of lime, lime z..."
12,Enchiladas Verdes (Green Enchiladas),['Mexican'],"[Meat, Produce;Ethnic Foods;Spices and Season...","[skinless boneless chicken breast, chili pepp..."
45,Salsa Verde Chicken Tamales,['Mexican'],"[Meat, Meat, Produce, Produce, Spices and ...","[roasted chicken, chicken, garlic, onion, ..."
89,Mexican Casserole,['Mexican'],"[Meat, Cheese, Canned and Jarred, Canned an...","[chicken breasts, colby jack, cream of chick..."
